In [141]:
import nltk
import math

def generate_tficf(reviews, n_cluster = 5):
    tf_word_freq = {}
    for n in range(n_cluster):
        tf_word_freq[n] = {}
   
    tf = []
    for index, review in enumerate(reviews):
        for sentence in nltk.sent_tokenize(review):
            for word in nltk.word_tokenize(sentence):
                if word in tf_word_freq[index]:
                    tf_word_freq[index][word] += 1
                else:
                    tf_word_freq[index][word] = 1

                if word not in tf:
                    tf.append(word)
                    
                    
    tf_icf = []
    for term in tf:
        tf_in_c = {}
        c_i = 0


        for n in range(n_cluster):
            tf_in_c[n] = 0
            
            if term in tf_word_freq[n]:
                tf_in_c[n] = tf_word_freq[n][term]
                c_i += 1


        icf = 1 + math.log(6/c_i)
        t = ()
        t += (term,)
        for n in range(n_cluster):
            t += (tf_in_c[n]*icf, )
        tf_icf.append(t)
    
    return tf_icf

In [142]:
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer 

lemmatizer = WordNetLemmatizer() 

nltk_words = set(nltk.corpus.stopwords.words('english'))
stop_words = []
for word in nltk_words:
    stop_words.append(word.translate(str.maketrans('', '', string.punctuation)))

def preprocess(sentence):
    res = sentence.lower()
    res = res.translate(str.maketrans('', '', string.punctuation))
    tokenized_words = nltk.word_tokenize(res)
    res = [word for word in tokenized_words if word not in stop_words]
    res = [lemmatizer.lemmatize(r) for r in res]
    res = [re.sub(r"[^A-Za-z]+", '', r) for r in res]
    res = [r for r in res if len(r) > 3]
    return ' '.join(res)

In [143]:
def open_file(kategori):
    f = open('wiki/source/' + kategori + '-sentence.txt', 'r', encoding='utf-8')
    all_sentences = []
    for line in f:
        all_sentences.append(preprocess(str(line).replace('\n','')))
    return ' '.join(all_sentences)

In [144]:
def gabung_file(daftar):
    res = []
    for judul in daftar:
        res.append(open_file(judul))
    return ' '.join(res)

In [145]:
ambience = gabung_file(['Theme restaurant', 'Atmosphere (architecture and spatial design)', 'Ambience (sound recording)'])

food = gabung_file(['Food', 'Drink', 'Meal'])

service = gabung_file(['Customer service', "Waiting staff"])

price = gabung_file(['Price', 'Pricing'])

In [146]:
d = generate_tficf([ambience, food, service, price], 4)

def get_keyword(index):
    sorted_word = sorted(d, key=lambda value: value[index], reverse=True)
    keyword = sorted_word[0:10]
    max_k = keyword[0][index]
    min_k = keyword[9][index]
    keyword = list(map(lambda value: (value[0], (value[1] - min_k) / (max_k - min_k)), keyword))
    return keyword

In [153]:
import pickle
def run(name, index):
    all_keyword = []
    for a, b in get_keyword(index):
        all_keyword.append(a)
    pickle_out = open("term/" + name + ".pickle","wb")
    pickle.dump(all_keyword, pickle_out)
    pickle_out.close()

In [154]:
data = [('ambience', 1), ('food', 2), ('service', 3), ('price', 4)]

In [155]:
for name, index in data:
    run(name, index)